2.1.3 Multigrid and Multilevel Methods
===

Multigrid (MG) and Multilevel (ML) algorithms provide preconditioners with optimal condition numbers $\kappa (C^{-1} A) = O(1)$, and optimal computational complexity $O(N)$.

They can be seen as extension of the two level overlapping domain decomposition method to more levels.

* Ulrich Trottenberg, Cornelius W. Oosterlee, Anton Schuller: Multigrid, Academic Press, 2001
* Wolfgang Hackbusch: Multi-Grid Methods and Applications, Springer, 1985

In short, both methods are sub-space correction methods where the space splitting is defined by all basis functions together from a hierarchy of grids. Multilevel methods are additive Schwarz methods, while Multigrid methods are the multiplicative Schwarz methods. Both can be implemented efficiently by recursive algorithms. 

We will present different theories for their analysis, one is based on sub-space decomposition using the ASM lemma, the other one uses the smoothing-and-approximation properties.

Multilevel preconditioner
---

We have a sequence of hierarchically refined meshes, which lead to a sequence of nested finite element spaces

$$
V_0 \subset V_1 \subset \ldots V_L
$$

of dimension $N_l = \operatorname{dim} V_l, l = 0 \ldots L$.
We think of mesh-sizes $h_l = 2^{-l}$, leading to finite element space dimensions $N^{dl}$, where $d$ is the spatial dimension.

We have prolongation matrices 

$$
P_l \in {\mathbb R}^{N_l \times N_{l-1}}.
$$

If $v_{l-1}$ is a finite element function in $V_{l-1}$ represented by the coefficient vector $\underline v_{l-1}$. Then
$\underline v_l = P_l \underline v_{l-1}$ is the coefficient vector of the same function represented by the basis functions of $V_l$.

If $A_l$ and $A_{l-1}$ are discretization matrices by a Galerkin method, then there holds

$$
A_{l-1} = P_l^T A_l P_l
$$

Let $D_l = \operatorname{diag} A_l$ be the Jacobi preconditioner (or some similar, cheap and local preconditioner).


**2-level method**

A 2-level preconditioner involving levels $l-1$ and level $l$ is

$$
C_{2L}^{-1} = D_l^{-1} + P_l A_{l-1}^{-1} P_l^T
$$

By the ASM - Lemma we have the tools to analyze the this preconditioner has optimal condition number. However, a direct inversion of the matrix $A_{l-1}$ is up to constant factor as expensive as the inversion of $A_l$. The multilevel method is to replace the inverses recursively by a multilevel preconditioner:

\begin{eqnarray*}
C_{ML,0}^{-1} & = & A_0^{-1} \\
C_{ML,l}^{-1} & = & D_l^{-1} + P_l C_{ML,l-1}^{-1} P_l^T \qquad \text{for} \; l = 1, \ldots L
\end{eqnarray*}

In [1]:
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.la import EigenValues_Preconditioner

mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

fes = H1(mesh,order=1, dirichlet=".*", autoupdate=True)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx)

In [2]:
class MLPreconditioner(BaseMatrix):
    def __init__ (self, fes, level, mat, coarsepre):
        super().__init__()
        self.fes = fes
        self.level = level
        self.mat = mat
        self.coarsepre = coarsepre
        if level > 0:
            self.localpre = mat.CreateSmoother(fes.FreeDofs())
        else:
            self.localpre = mat.Inverse(fes.FreeDofs())
        
    def Mult (self, x, y):
        if self.level == 0:
            y.data = self.localpre * x
            return
        hx = x.CreateVector(copy=True)
        self.fes.Prolongation().Restrict(self.level, hx)
        cdofs = self.fes.Prolongation().LevelDofs(self.level-1)
        y[cdofs] = self.coarsepre * hx[cdofs] 
        self.fes.Prolongation().Prolongate(self.level, y)
        y += self.localpre * x

    def Shape (self):
        return self.localpre.shape
    def CreateVector (self, col):
        return self.localpre.CreateVector(col)

With operator notation we can define the multilevel preconditioner also as follows:

In [3]:
def MLPreconditioner2(fes, level, mat, coarsepre):
    prol = fes.Prolongation().Operator(level)
    localpre = mat.CreateSmoother(fes.FreeDofs())
    return localpre + prol @ coarsepre @ prol.T

In [ ]:
a.Assemble()
pre = a.mat.Inverse(fes.FreeDofs())

In [ ]:
for l in range(8):
    mesh.Refine()
    print ("ndof = ", fes.ndof)
    a.Assemble()
    pre = MLPreconditioner(fes,l+1, a.mat, pre)
    
    lam = EigenValues_Preconditioner(a.mat, pre)
    print ("lammin, lammax=", lam[0], lam[-1], \
            "kappa=", lam[-1]/lam[0])

Multigrid Preconditioning
---

The multigrid preconditioner combines the local preconditioner and the coarse grid step sequentially. The multigrid preconditioning action is define as follows:

Multigrid $C_{MG,l}^{-1} : d \mapsto w$

* if l = 0, set $w = A_0^{-1} d$ and return
* w = 0
* presmoothing, $m_l$ steps:   
$$\hat w = w + D_{pre}^{-1} (d - A w)$$ 
* coasre grid correction:
$$
\hat w = w + P_l C_{MG,l-1}^{-1} P_l^{T} (d - A w)
$$
* postsmoothing, $m_l$ steps:   
$$\hat w = w + D_{post}^{-1} (d - A w)$$ 


If the preconditioners $D_{pre}$ and $D_{post}$ from the pre-smoothing and post-smoothing iterations are transposed to each other, the overall preconditioner is symmetric. If the pre- and post-smoothing iterations are non-expansive, the overall preconditioner is positive definite.
This can be obtained by applying forward Gauss-Seidel for pre-smoothing, and backward Gauss-Seidel for post-smoothing.

In [ ]:
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.la import EigenValues_Preconditioner

mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

fes = H1(mesh,order=1, dirichlet=".*")
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx)

In [ ]:
class MGPreconditioner(BaseMatrix):
    def __init__ (self, fes, level, mat, coarsepre):
        super().__init__()
        self.fes = fes
        self.level = level
        self.mat = mat
        self.coarsepre = coarsepre
        if level > 0:
            self.localpre = mat.CreateSmoother(fes.FreeDofs())
        else:
            self.localpre = mat.Inverse(fes.FreeDofs())
        
    def Mult (self, d, w):
        # print ("level = ", self.level)
        if self.level == 0:
            w.data = self.localpre * d
            return
        
        prol = self.fes.Prolongation().Operator(self.level)

        w[:] = 0
        self.localpre.Smooth(w,d)
        res  = d - self.mat * w
        w += prol @ self.coarsepre @ prol.T * res
        self.localpre.SmoothBack(w,d)


    def Shape (self):
        return self.localpre.shape
    def CreateVector (self, col):
        return self.localpre.CreateVector(col)

In [ ]:
a.Assemble()
pre = MGPreconditioner(fes, 0, a.mat, None)

for l in range(6):
    mesh.Refine()
    fes.Update()
    print ("ndof = ", fes.ndof)
    a.Assemble()
    pre = MGPreconditioner(fes,l+1, a.mat, pre)
    
    lam = EigenValues_Preconditioner(a.mat, pre)
    print ("lammin, lammax=", lam[0], lam[-1], \
            "kappa=", lam[-1]/lam[0])

In [ ]:
f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)
from ngsolve.krylovspace import CGSolver
inv = CGSolver(mat=a.mat, pre=pre, printrates=True)
gfu.vec.data = inv * f.vec

In [ ]:
from ngsolve.webgui import Draw
if fes.ndof < 100000:
    Draw(gfu)

Projection matrices from the finest level
---
It is often not feasible to assemble matrices on the coarse level, 
for example when solving non-linear problems. Then it is useful to 
calculate coarse grid matrices from the matrix on the finest level using the Galerkin property

$$
A_{l-1} = P_{l}^T A_l P_l
$$

\[ Requires NGSolve updates from 14.04.2021, still tricky: freedofs on coarser levels]

In [ ]:
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.la import EigenValues_Preconditioner

mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

fes = H1(mesh,order=1, dirichlet=".*", autoupdate=True)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx)

for l in range(6):
     mesh.Refine()
    
a.Assemble()

In [ ]:
class ProjectedMG(BaseMatrix):
    def __init__ (self, fes, mat, level):
        super(ProjectedMG, self).__init__()
        self.fes = fes
        self.level = level
        self.mat = mat
        if level > 0:
            self.prol = fes.Prolongation().CreateMatrix(level)
            self.rest = self.prol.CreateTranspose()
            coarsemat = self.rest @ mat @ self.prol # multiply matrices
            self.localpre = mat.CreateSmoother(fes.FreeDofs())
                
            self.coarsepre = ProjectedMG(fes, coarsemat, level-1)
        else:
            self.localpre = mat.Inverse(fes.FreeDofs())
        
    def Mult (self, d, w):
        if self.level == 0:
            w.data = self.localpre * d
            return
        w[:] = 0
        self.localpre.Smooth(w,d)
        res = d - self.mat * w
        w += self.prol @ self.coarsepre @ self.rest * res
        self.localpre.SmoothBack(w,d)

        
    def Shape (self):
        return self.localpre.shape
    def CreateVector (self, col):
        return self.localpre.CreateVector(col)

In [ ]:
# print (fes.mesh.levels)
pre = ProjectedMG(fes, a.mat, fes.mesh.levels-1)

In [ ]:
print ("ndof: ", fes.ndof)
lam = EigenValues_Preconditioner(a.mat, pre)
print ("lammin, lammax=", lam[0], lam[-1], \
            "kappa=", lam[-1]/lam[0])

In [ ]:
f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)
from ngsolve.krylovspace import CGSolver
inv = CGSolver(mat=a.mat, pre=pre, printrates=True)
gfu.vec.data = inv * f.vec

from ngsolve.webgui import Draw
if fes.ndof < 100000:
    Draw(gfu, order=1)